Name : Prem Vikas Palkar
College : KIT College of Engineering Kolahpur.
Year : 3rd Year
Branch : CSE(AIML)

In [8]:
#  Importing required Libraries 
import pandas as pd
import sqlite3 as sql

# Load csv data
orders = pd.read_csv("orders.csv")

# load json data
users = pd.read_json("users.json")

In [20]:
# Establish a connection to the SQLite database and read the restaurants.sql file

conn = sql.connect("restaurant.db")
cursor = conn.cursor()
with open("restaurants.sql", "r", encoding = "utf-8") as  f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()

OperationalError: table restaurants already exists

In [259]:
# Load SQL data
restaurants = pd.read_sql("select * from restaurants;", conn)
restaurants.head()
conn.close()

In [24]:
orders.shape

(10000, 6)

In [25]:
users.shape

(3000, 4)

In [27]:
restaurants.shape

(500, 4)

In [29]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   order_id         10000 non-null  int64  
 1   user_id          10000 non-null  int64  
 2   restaurant_id    10000 non-null  int64  
 3   order_date       10000 non-null  object 
 4   total_amount     10000 non-null  float64
 5   restaurant_name  10000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 468.9+ KB


In [45]:
# I have created a function to quicky check the shape and its mising values of datasets before merging
def inspect(data):
    print("Shape : ",data.shape)
    print("Null value Count : \n" ,data.isnull().sum())

In [46]:
inspect(orders)

Shape :  (10000, 6)
Null value Count : 
 order_id           0
user_id            0
restaurant_id      0
order_date         0
total_amount       0
restaurant_name    0
dtype: int64


In [48]:
inspect(users)

Shape :  (3000, 4)
Null value Count : 
 user_id       0
name          0
city          0
membership    0
dtype: int64


In [49]:
inspect(restaurants)

Shape :  (500, 4)
Null value Count : 
 restaurant_id      0
restaurant_name    0
cuisine            0
rating             0
dtype: int64


In [37]:
# Merging Data

# Merge 1, users + orders
orders_users = pd.merge(orders, users, how = 'left', on = 'user_id')

# Merge 2, users + orders + restaurant
final_df = pd.merge(orders_users, restaurants, how ='left', on='restaurant_id')

In [53]:
# Check whether merged data and orders data have equal rows

print("Orders Shape: ",orders.shape)
print("Final Dataset Shape: ",final_df.shape)

Orders Shape:  (10000, 6)
Final Dataset Shape:  (10000, 12)


In [43]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


# 1. Highest total revenue (total_amount) from Gold members

In [262]:
gold_df = final_df[final_df['membership'] == 'Gold']
city_revenue = gold_df.groupby('city')['total_amount'].sum()
city_revenue

city
Bangalore     994702.59
Chennai      1080909.79
Hyderabad     896740.19
Pune         1003012.32
Name: total_amount, dtype: float64

In [ ]:
# Chennai has the highest total revenue from Gold members.

## 2. Which cuisine has the highest average order value across all orders

In [69]:
cuisine_df = final_df.groupby('cuisine')['total_amount'].mean()
cuisine_df

cuisine
Chinese    798.389020
Indian     798.466011
Italian    799.448578
Mexican    808.021344
Name: total_amount, dtype: float64

In [70]:
# Mexican cuisine has the highest average order value across all orders.

## 3. How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)

In [81]:
user_total = final_df.groupby('user_id')['total_amount'].sum()
high_value_users = user_total[user_total >1000]
high_value_users.count()

np.int64(2544)

In [82]:
# There are 2544 distinct users placed orders worth more than 1000 in total.

## 4. Which restaurant rating range generated the highest total revenue?

In [263]:
# Using binning for creating rating range
final_df['rating_range'] = pd.cut(final_df['rating'],
                                 bins = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0],
                                 labels =['1.0-1.5', '1.6-2.0', '2.1-2.5', '2.6-3.0', '3.1-3.5', '3.6-4.0', '4.1-4.5', '4.6-5.0'],
                                 include_lowest =True)

In [95]:
final_df[['rating','rating_range']].head()

,rating,rating_range
0,3.2,3.1-3.5
1,4.5,4.1-4.5
2,4.0,3.6-4.0
3,4.8,4.6-5.0
4,3.0,2.6-3.0


In [94]:
restaurant_rating_revenue = final_df.groupby('rating_range')['total_amount'].sum()
restaurant_rating_revenue.idxmax()

C:\Users\premv\AppData\Local\Temp\ipykernel_17964\1125864117.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  restaurant_rating_revenue = final_df.groupby('rating_range')['total_amount'].sum()


'4.6-5.0'

In [ ]:
# '4.6 -5.0' restaurant rating range generated the highest total revenue.

## 5. Among Gold members, which city has the highest average order value?

In [264]:
city_avg_order_val = gold_df.groupby('city')['total_amount'].mean()
city_avg_order_val

city
Bangalore    793.223756
Chennai      808.459080
Hyderabad    806.421034
Pune         781.162243
Name: total_amount, dtype: float64

In [265]:
city_avg_order_val.idxmax()

'Chennai'

In [102]:
#  Chennai has the highest average order value among Gold members.

## 6. Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?

In [104]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating,rating_range
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2,3.1-3.5
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5,4.1-4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0,3.6-4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8,4.6-5.0
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0,2.6-3.0


In [114]:
cuisine_restaurant_count = final_df.groupby('cuisine')['restaurant_id'].nunique()
cuisine_restaurant_count.sort_values()

cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64

In [126]:
# Total Revenue per cuisine
cuisine_revenue = final_df.groupby('cuisine')['total_amount'].sum()

# Join unique restaurant and cuisine revenue for comparison
cuisine_summary = pd.concat([cuisine_restaurant_count,cuisine_revenue], axis = 1)
cuisine_summary.columns=['rest_count','total_revenue']
cuisine_summary

,rest_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Indian,126,1971412.58
Italian,126,2024203.80
Mexican,128,2085503.09


In [127]:
# Chinese cuisine has the lowest number of distinct restaurants but still contributes significant revenue

## 7. What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [144]:
total_orders = len(final_df)
total_gold_orders = len(gold_df)                             # gold_df already declared at the 1st question
gold_order_percentage = total_gold_orders/total_orders*100   # using percentage formula
round(gold_order_percentage)

50

In [145]:
# 50 % of total orders were placed by Gold members!

## 8. Which restaurant has the highest average order value but less than 20 total orders?

In [147]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating,rating_range
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2,3.1-3.5
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5,4.1-4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0,3.6-4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8,4.6-5.0
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0,2.6-3.0


In [197]:
rest_avg_orders = final_df.groupby('restaurant_name_x').agg( total_orders = ('order_id', 'count'),
                                                           avg_order_value = ('total_amount', 'mean'))
rest_avg_orders.head()

,total_orders,avg_order_value
restaurant_name_x,,
Amma Biryani Family Restaurant,45,799.941556
Amma Biryani North Indian,57,774.900526
Amma Biryani South Indian,15,708.708667
Amma Cafe Non-Veg,24,736.174583
Amma Cafe North Indian,11,667.979091


In [198]:
low_order_restaurants = rest_avg_orders[rest_avg_orders['total_orders'] < 20]
low_order_restaurants.head()

,total_orders,avg_order_value
restaurant_name_x,,
Amma Biryani South Indian,15,708.708667
Amma Cafe North Indian,11,667.979091
Amma Cafe Pure Veg,17,839.633529
Amma Curry House Family Restaurant,18,821.800556
Amma Delights Family Restaurant,17,761.249412


In [199]:
low_order_restaurants['avg_order_value'].idxmax()

'Hotel Dhaba Multicuisine'

In [193]:
#  From overall dataset it shows that 'Hotel Dhaba Multicuisine' has the highest average.(not in options) 

In [272]:
options = [
    'Grand Cafe Punjabi',
    'Grand Restaurant South Indian',
    'Ruchi Mess Multicuisine',
    'Ruchi Foods Chinese'
]


option_df = final_df[final_df['restaurant_name_x'].isin(options)]

option_df = option_df.groupby('restaurant_name_x').agg(
    total_orders=('order_id', 'count'),
    avg_order_value=('total_amount', 'mean')
)
low_order_options = option_df[option_df['total_orders'] < 20]
low_order_options


,total_orders,avg_order_value
restaurant_name_x,,
Ruchi Foods Chinese,19,686.603158


In [ ]:
# Looks like only Ruchi Foods Chinese has the total orders less than 20, SO from options of question 8, "Ruchi Foods Chinese" is the right option.

## 9. Which combination contributes the highest revenue?

In [258]:
# Here I created a function to get combined revenue for any membership and cuisine.

def combination_revenue(membership, cuisine):
    filtered_df = final_df[(final_df['membership'] == membership) & (final_df['cuisine'] == cuisine)]
    print(f"{membership} + {cuisine} , Combined Revenue: {filtered_df['total_amount'].sum()}")

combination_revenue("Gold", "Indian")
combination_revenue("Gold", "Italian")
combination_revenue("Regular", "Indian")
combination_revenue("Regular", "Chinese")

Gold + Indian , Combined Revenue: 979312.31
Gold + Italian , Combined Revenue: 1005779.05
Regular + Indian , Combined Revenue: 992100.27
Regular + Chinese , Combined Revenue: 952790.9099999999


In [ ]:
# Gold + Italian combination contributes the highest revenue i.e 1005779.05

## 10. During which quarter of the year is the total revenue highest?

In [200]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating,rating_range
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2,3.1-3.5
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5,4.1-4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0,3.6-4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8,4.6-5.0
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0,2.6-3.0


In [202]:
final_df['order_date'].dtype

dtype('O')

In [204]:
final_df['order_date'] = pd.to_datetime(final_df['order_date'])

C:\Users\premv\AppData\Local\Temp\ipykernel_17964\606004383.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df['order_date'] = pd.to_datetime(final_df['order_date'])


In [207]:
final_df['quarter'] = final_df['order_date'].dt.quarter

In [210]:
quarter_revenue = final_df.groupby('quarter')['total_amount'].sum()

In [212]:
quarter_revenue.idxmax()

np.int32(3)

In [ ]:
# Q3 (July–September) has the highest total revenue.

## #Numerical Answers

In [213]:
# How many total orders were placed by users with Gold membership?
gold_df.shape[0]

4987

In [216]:
# What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?

city_revenues = final_df.groupby('city')['total_amount'].sum()
round(city_revenues)

city
Bangalore    2206947.0
Chennai      1990513.0
Hyderabad    1889367.0
Pune         1924798.0
Name: total_amount, dtype: float64

In [217]:
# How many distinct users placed at least one order?
final_df['user_id'].nunique()

2883

In [219]:
round(gold_df['total_amount'].mean(),2)

np.float64(797.15)

In [223]:
# How many orders were placed for restaurants with rating ≥ 4.5?
final_df[final_df['rating'] >= 4.5].shape[0]

3374

In [236]:
# How many orders were placed in the top revenue city among Gold members only?
gold_df.groupby('city')['total_amount'].sum().idxmax()

'Chennai'

In [237]:
# Chennai is the top revenue city
gold_df[gold_df['city'] == 'Chennai'].shape[0] #to calculate the total orders

1337

In [238]:
final_df.shape

(10000, 14)

In [239]:
# Exporting final dataset
final_df.to_csv("final_food_delivery_dataset.csv", index=False)